In [1]:
def show_user(string : str) :
    print("Clients chat vew : ",string)
    return

In [1]:
import secrets
secret_key = secrets.token_hex(32)  # Generates a random 64-character hex string
print(secret_key)


e867970cb91e811bc4fe9accab3cb8ce87f89aef542b8167deba741ae9499f85


In [2]:

#  define a place to store the chat history and initialize it as an empty list and set and get the chat history
chat_history = []


def add_to_chat_history(message: str):
    global chat_history
    chat_history.append(message)
    # only keep the last 5 messages in the chat history
    chat_history = chat_history[-5:]
    return chat_history

def get_chat_history():
    global chat_history
    return chat_history

# create a hash table to store serach items history
search_history = {}

def add_to_search_history(search_index: int, search_item: str):
    global search_history
    search_history[search_index] = search_item
    return search_history

def get_search_history_item(search_index: int):
    global search_history
    return search_history[search_index]

def get_search_history():
    global search_history
    return search_history

def delete_search_history_item(search_index: int):
    global search_history
    del search_history[search_index]
    return search_history

def clear_search_history():
    global search_history
    search_history.clear()
    return search_history

In [3]:
from openai import OpenAI
client = OpenAI()

# Define the function with chat history memory
def openAI_chat_assistant(new_message: str,tools= None,response_format=None ,  temperature: float = 0, max_tokens: int = 1024 ) :
    # Initialize 
    prompt = [{"role": "system", "content": "You are a helpful shopping assistant. Answer in very short and simple sentences. Dont ask too many questions, and don't respond with un provided information."}]
    
    chat_history = get_chat_history()
    
    chat_history = add_to_chat_history({"role": "user", "content": new_message})
    
    prompt.extend(chat_history) if chat_history else prompt
    
    # Call the OpenAI API to generate a response
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=prompt,
        temperature=temperature,
        max_tokens=max_tokens,
        tools=tools,
        response_format = response_format
    )
    
    # Extract the assistant's response
    assistant_reply = completion.choices[0].message.content

    
    # Return the reply and updated chat history
    return completion

In [4]:
import json
from pydantic import BaseModel
from openai import OpenAI
client = OpenAI()

def order_the_best_5_search_results(best_five_results: str):
    # Parse the input JSON string
    best_five_results = json.loads(best_five_results)
    
    # Extract the search index from the parsed JSON
    search_index = best_five_results.get("search_index", "Unknown Search Index")
    
    # Extract product details from the results
    product_attributes = []
    for product in best_five_results.get("results", []):
        product_attributes.append({
            "product_id": product["product_id"],
            "title": product["title"],
            "price": product["price"],
            "extracted_price": product["extracted_price"],
            "rating": product["rating"],
            "reviews": product["reviews"],
            "seller": product["seller"],
            "delivery": product["delivery"],
            "brand": product["brand"],
            "extensions": product["extensions"]
        })
    
    # Combine the search index and product attributes into a final JSON object
    result_json = {
        "search_index": search_index,
        "products": product_attributes
    }
    
    # Convert the final JSON object to a string
    result_json_string = json.dumps(result_json, indent=4)
    
    user_searched_item = get_search_history_item(search_index)
    
    users_quarry = user_searched_item.get("users_quarry")
    
    # print("User searched quarry: ", users_quarry)
    
    quarry = users_quarry + " These are the attributes of the item user searched for , according to that , rank the following 5 search results to mach the users search criteria and to be the overall best product from the search results considering all the factors like users needs, price, rating, reviews, and other features. keep your reasoning very short , and remember you are giving this reasoning directly to the user so use proper pronounce. 5 Search results : "+ result_json_string
    
    class OrderStructured(BaseModel):
        reasons_for_the_order: str
        search_result_rank_1_product_id: str
        search_result_rank_2_product_id: str
        search_result_rank_3_product_id: str
        search_result_rank_4_product_id: str
        search_result_rank_5_product_id: str
        
       
    completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": quarry},
        ],
        response_format=OrderStructured,
    )
        
    
    json_gpt_ranking = completion.choices[0].message.content
    
    # print(json_gpt_ranking)
    
        # Parse the GPT response for the ranking information
    ranking_info = json.loads(json_gpt_ranking)
    
    # Map product IDs to their ranks
    product_rank_map = {
        ranking_info["search_result_rank_1_product_id"]: 1,
        ranking_info["search_result_rank_2_product_id"]: 2,
        ranking_info["search_result_rank_3_product_id"]: 3,
        ranking_info["search_result_rank_4_product_id"]: 4,
        ranking_info["search_result_rank_5_product_id"]: 5,
    }
    
    # Add rank and reasoning to each product in the result
    reasoning = ranking_info["reasons_for_the_order"]
    
    best_five_results["rank_reasoning"] = reasoning
    
    for results in best_five_results["results"]:
        results["rank"] = product_rank_map.get(results["product_id"], "Not Ranked")

    # Convert the final result to a JSON string with ranks and reasoning
    result_json_string_with_ranking = json.dumps(best_five_results, indent=4)
    
    print("final result with ranking and reasoning: ", result_json_string_with_ranking)
    
    add_to_chat_history({"role": "assistant", "content": result_json_string_with_ranking})
    
    return result_json_string_with_ranking


In [5]:
import json


def talk_to_gpt(user_input : str):
  chat_history = get_chat_history()
  
  tools = [
    {
        "type": "function",
        "function": {
    "name": "search_item",
    "description": "Search for an item to buy from an online store based on various criteria.if the user has provided them. if user has only provided a few continue with only what they have given.Do not ask him to give more details continually if he has not provided them.",
    "strict": True,
    "parameters": {
      "type": "object",
      "required": [
        "users_quarry",
        "item",
        "category",
        "store",
        "brand",
        "min_price",
        "max_price",
        "min_rating",
        "features",
        "keywords",
        "material",
        "color",
        "size_dimension",
        "size_scale"
      ],
      "properties": {
        "users_quarry": {
          "type": "string",
          "description": "summarized shorten version of the users exact search request"
        },
        "item": {
          "type": "string",
          "description": "The name of the item being searched. Must have a value."
        },
        "category": {
          "type": "string",
          "description": "The category of the item"
        },
        "store": {
          "type": "string",
          "description": "The online store to search in, select 'Not Specified' if user haven't mention a specific store",
          "enum": [
            "Not Specified",
            "Amazon",
            "eBay",
            "Walmart",
            "Target",
            "Best Buy",
            "Etsy",
            "Home Depot",
            "Costco",
            "Wayfair",
            "Chewy",
            "Macy's",
            "Newegg",
            "Kohl's",
            "Nordstrom",
            "Overstock"
          ]
        },
        "brand": {
          "type": "string",
          "description": "The brand of the item, leave null if not specified"
        },
        "min_price": {
          "type": "number",
          "description": "Minimum price of the item, leave null if not specified"
        },
        "max_price": {
          "type": "number",
          "description": "Maximum price of the item, leave null if not specified"
        },
        "min_rating": {
          "type": "number",
          "description": "Minimum rating of the item (1 to 5), leave null if not specified"
        },
        "features": {
          "type": "array",
          "description": "List of features the item should have",
          "items": {
            "type": "string",
            "description": "Feature of the item"
          }
        },
        "keywords": {
          "type": "array",
          "description": "List of keywords related to the search",
          "items": {
            "type": "string",
            "description": "Keyword for the search"
          }
        },
        "material": {
          "type": "string",
          "description": "Material of the item"
        },
        "color": {
          "type": "string",
          "description": "Color of the item"
        },
        "size_dimension": {
          "type": "integer",
          "description": "Size of the item, provide a value in this only if the size is in inches or cm or any other unit is provided."
        },
        "size_scale": {
          "type": "string",
          "description": "Relative size, e.g., small, medium, or large.",
          "enum": [
            "Not Specified",
            "xxs",
            "xs",
            "s",
            "m",
            "l",
            "xl",
            "xxl"
          ]
        }
      },
      "additionalProperties": False
    }
  }

    }
  ]

  response = openAI_chat_assistant(user_input,tools=tools)
  response_msg = response.choices[0].message.content
  function_response_msg = None

  # if there is a function call, extract the function call information
  response_func = None
  if (response.choices[0].message.tool_calls):
      response_func = response.choices[0].message.tool_calls[0].function.name
      
      
      # write a switch case to call the function with the arguments
      if response_func == "search_item":
          arguments = response.choices[0].message.tool_calls[0].function.arguments
          # print("Passing arguments to search_item function: ",arguments )
          # argument is a string in json format, so we need to parse it
          function_response_msg = search_item(arguments)
          
  return_msg = ""

  if response_msg and function_response_msg:
      return_msg = response_msg + function_response_msg
  elif function_response_msg:
      return_msg = function_response_msg
  elif response_msg:
      return_msg = response_msg 
      
  chat_history= add_to_chat_history({"role": "assistant", "content": return_msg})

  show_user(return_msg)
  print(chat_history)

search_id = 0
def search_item(attributes_json: str) :
    global search_id
    search_id += 1
    arguments = json.loads(attributes_json)
    print("attributes has been received", attributes_json)
    add_to_search_history(search_id, arguments)
    
    return "Searching the web for the item"

In [6]:
talk_to_gpt("I want to buy a red dress, medium size, under $50, from Amazon, It would be great if it is from a good brand and has good reviews")

Passing arguments to search_item function:  {"users_quarry":"buy a red dress","item":"dress","category":"clothing","store":"Amazon","brand":"" ,"min_price":0,"max_price":50,"min_rating":4,"features":[],"keywords":[],"material":"","color":"red","size_dimension":0,"size_scale":"m"}
attributes has been received {"users_quarry":"buy a red dress","item":"dress","category":"clothing","store":"Amazon","brand":"" ,"min_price":0,"max_price":50,"min_rating":4,"features":[],"keywords":[],"material":"","color":"red","size_dimension":0,"size_scale":"m"}
Searching the web for the item
[{'role': 'user', 'content': 'I want to buy a red dress, medium size, under $50, from Amazon, It would be great if it is from a good brand and has good reviews'}, {'role': 'assistant', 'content': 'Searching the web for the item'}]


In [7]:
somple_top_search_results= {
  "search_index": 1,
  "results": [
    {
      "position": 1,
      "product_id": "11234567890123456789",
      "title": "ECOWISH Women's Casual V Neck Button Down Red Dress - Medium",
      "link": "https://www.amazon.com/dp/B089W7MXYZ",
      "product_link": "https://www.amazon.com/dp/B089W7MXYZ",
      "seller": "Amazon",
      "price": "$39.99",
      "extracted_price": 39.99,
      "rating": 4.5,
      "reviews": 1532,
      "delivery": "Free delivery for Prime members",
      "extensions": ["V Neck", "Short Sleeve", "Knee-Length"],
      "brand": "ECOWISH",
      "thumbnail": "https://example.com/images/red_dress1.jpg"
    },
    {
      "position": 2,
      "product_id": "22345678901234567890",
      "title": "ANRABESS Women's Summer Casual Sundress - Red, Medium",
      "link": "https://www.amazon.com/dp/B093J6ABCD",
      "product_link": "https://www.amazon.com/dp/B093J6ABCD",
      "seller": "Amazon",
      "price": "$29.99",
      "extracted_price": 29.99,
      "rating": 4.7,
      "reviews": 2280,
      "delivery": "Free delivery with $25 order",
      "extensions": ["Spaghetti Strap", "Flowy", "Lightweight"],
      "brand": "ANRABESS",
      "thumbnail": "https://example.com/images/red_dress2.jpg"
    },
    {
      "position": 3,
      "product_id": "33456789012345678901",
      "title": "MEROKEETY Women's Sleeveless Wrap Dress - Red, Medium",
      "link": "https://www.amazon.com/dp/B08KL7EFGH",
      "product_link": "https://www.amazon.com/dp/B08KL7EFGH",
      "seller": "Amazon",
      "price": "$47.99",
      "extracted_price": 47.99,
      "rating": 4.6,
      "reviews": 874,
      "delivery": "Free delivery for Prime members",
      "extensions": ["Wrap Style", "Adjustable Waist", "Knee-Length"],
      "brand": "MEROKEETY",
      "thumbnail": "https://example.com/images/red_dress3.jpg"
    },
    {
      "position": 4,
      "product_id": "44567890123456789012",
      "title": "PRETTYGARDEN Women's Casual T-Shirt Dress - Red, Medium",
      "link": "https://www.amazon.com/dp/B08KL7LMNO",
      "product_link": "https://www.amazon.com/dp/B08KL7LMNO",
      "seller": "Amazon",
      "price": "$34.99",
      "extracted_price": 34.99,
      "rating": 4.3,
      "reviews": 640,
      "delivery": "Free delivery with $25 order",
      "extensions": ["T-Shirt Style", "Soft Fabric", "Midi Length"],
      "brand": "PRETTYGARDEN",
      "thumbnail": "https://example.com/images/red_dress4.jpg"
    },
    {
      "position": 5,
      "product_id": "55678901234567890123",
      "title": "ZESICA Women's Summer Wrap Maxi Dress - Red, Medium",
      "link": "https://www.amazon.com/dp/B08KL7PQRS",
      "product_link": "https://www.amazon.com/dp/B08KL7PQRS",
      "seller": "Amazon",
      "price": "$45.99",
      "extracted_price": 45.99,
      "rating": 4.5,
      "reviews": 1325,
      "delivery": "Free delivery for Prime members",
      "extensions": ["Maxi Length", "Floral Design", "Bohemian Style"],
      "brand": "ZESICA",
      "thumbnail": "https://example.com/images/red_dress5.jpg"
    }
  ]
}



# Call the function with the converted JSON string
result = order_the_best_5_search_results(json.dumps(somple_top_search_results))

# Print the output
# print(result)

User searched quarry:  buy a red dress
{"reasons_for_the_order":"I ranked these products based on a combination of price, user ratings, number of reviews, and suitability to your search for a red dress. The first product is the best mix of high rating and lower price, while the second offers the best rating and is also affordable. The others are slightly higher in price or ratings but don't match the top two in overall value.","search_result_rank_1_product_id":"22345678901234567890","search_result_rank_2_product_id":"11234567890123456789","search_result_rank_3_product_id":"33456789012345678901","search_result_rank_4_product_id":"55678901234567890123","search_result_rank_5_product_id":"44567890123456789012"}
final result with ranking and reasoning:  {
    "search_index": 1,
    "results": [
        {
            "position": 1,
            "product_id": "11234567890123456789",
            "title": "ECOWISH Women's Casual V Neck Button Down Red Dress - Medium",
            "link": "https:

In [8]:
talk_to_gpt("can you explain me why you picked that dress for me?")

I selected dresses based on price, ratings, and reviews. The top options are affordable and have good ratings. They match your request for a red dress in medium size.
[{'role': 'user', 'content': 'I want to buy a red dress, medium size, under $50, from Amazon, It would be great if it is from a good brand and has good reviews'}, {'role': 'assistant', 'content': 'Searching the web for the item'}, {'role': 'assistant', 'content': '{\n    "search_index": 1,\n    "results": [\n        {\n            "position": 1,\n            "product_id": "11234567890123456789",\n            "title": "ECOWISH Women\'s Casual V Neck Button Down Red Dress - Medium",\n            "link": "https://www.amazon.com/dp/B089W7MXYZ",\n            "product_link": "https://www.amazon.com/dp/B089W7MXYZ",\n            "seller": "Amazon",\n            "price": "$39.99",\n            "extracted_price": 39.99,\n            "rating": 4.5,\n            "reviews": 1532,\n            "delivery": "Free delivery for Prime membe

In [9]:
talk_to_gpt("can you send me the link to the best one u picked")

Here is the link to the best dress I picked:

[ANRABESS Women's Summer Casual Sundress - Red, Medium](https://www.amazon.com/dp/B093J6ABCD) - Price: $29.99, Rating: 4.7.
[{'role': 'assistant', 'content': '{\n    "search_index": 1,\n    "results": [\n        {\n            "position": 1,\n            "product_id": "11234567890123456789",\n            "title": "ECOWISH Women\'s Casual V Neck Button Down Red Dress - Medium",\n            "link": "https://www.amazon.com/dp/B089W7MXYZ",\n            "product_link": "https://www.amazon.com/dp/B089W7MXYZ",\n            "seller": "Amazon",\n            "price": "$39.99",\n            "extracted_price": 39.99,\n            "rating": 4.5,\n            "reviews": 1532,\n            "delivery": "Free delivery for Prime members",\n            "extensions": [\n                "V Neck",\n                "Short Sleeve",\n                "Knee-Length"\n            ],\n            "brand": "ECOWISH",\n            "thumbnail": "https://example.com/images